# Twitch data analysis

Dataset nodi (streamer):
- numero persone totali raggiunte
- tempo streammato
- viewer medi
- aggiornamento community (persone totali/tempo)/viewer medi
- peak viewers
- categorie streammate
- link twitter
- dati twitter

Dataset relazioni/ matrice streamer:
- percentuale
- numero assoluto

#### Setup parameters

In [1]:
freq = 2               #Sample frequency (1=5min, 2=10min, 3=15min ecc.)
min_stream_time = 3    #Minimum live time parameter
min_watch_time = 3     #Minimum threshold of time as a spectator (min_watch_time <= min_stream_time)
min_num_spect_mean = 3 #Minimum number of spectator
interval = 5           #Interval between observations

print(f'Minimum live time threshold:\t\t\t {interval * freq * min_stream_time} min')
print(f'Minimum threshold of time as a spectator:\t {interval * freq * min_watch_time} min')

Minimum live time threshold:			 30 min
Minimum threshold of time as a spectator:	 30 min


#### Import libraries

In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize
from collections import Counter
from collections import defaultdict

## Nodes dataset

### Import data

Complete list of files

In [3]:
relpath1 = '..\DataCollection\Twitch_stream_spect\Files_stream\\'
completelist = [os.path.join(path, name) for path, subdirs, files in os.walk(relpath1) for name in files]
print("Number of files:", len(completelist))
print(*completelist[:5], sep = "\n")

Number of files: 278
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-04.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-09.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-14.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-19.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-24.json


Sampling files based on frequency

In [4]:
flist = completelist[::freq]
print("Number of files:", len(flist))
print(*flist[:5], sep = "\n")

Number of files: 139
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-04.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-14.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-24.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-34.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-06_00-44.json


Import selected files

In [5]:
data_dict = {}

for file in flist:
    start = relpath1
    end = '.json'
    time = file[len(start): - len(end)]
    time_obj = datetime.strptime(time, '%Y-%m-%d_%H-%M') #extract timestamp
    
    with open(file, 'r') as f:
        data_dict[time_obj] = json.load(f)

print("Number of keys (time intervals):", len(data_dict.keys()))

Number of keys (time intervals): 139


Example `'pasquale890': {'spect': ['anytow'], 'game_name': 'FIFA 22', 'viewer_count': 0}`


In [6]:
relpath2 = '..\DataCollection\Other_datasets\\'

bot_list = pd.read_csv(f'{relpath2}Twitch_bot_list.csv')
bot_list = bot_list['Twitch Username'].tolist()
bot_list[:10]

['turbopascai',
 'tteyyd',
 'twitchdetails',
 '0liviajadeee',
 'judgejudysiayer',
 'thelurkertv',
 'nothingbutgay',
 'streamers_on_discord',
 '0_applebadapple_0',
 'agressive_sock']

### Remove bot

Difference between viewer and spect for a time instant, can be negative and positive:

- bot excluded from viewer count
- viewer not connected in chat
- data updated at different instants

In [7]:
spect_with_bot = {}

for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    spect_with_bot[streamer] = len(data_dict[time_obj][streamer]['spect'])
    '''
    print(f"{streamer}\n\
    \tViewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
    \tSpect Count: {spect_with_bot[streamer]}\
    \tDifference: {data_dict[time_obj][streamer]['diff_count']}\
    ")
    '''

In [8]:
def remove_bot(spect_with_bot, bot_set):
    return list(set(spect_with_bot) - bot_set)

In [9]:
bot_set = set(bot_list)
for time_obj in data_dict.keys():
    for streamer in data_dict[time_obj].keys():
        data_dict[time_obj][streamer]['spect'] = remove_bot(data_dict[time_obj][streamer]['spect'], bot_set)

Check again the difference between viewer and spect

In [10]:
for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    
    print(f"{streamer}\n\
    \tViewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
    \tSpect Count: {len(data_dict[time_obj][streamer]['spect'])}\
    \tDifference: {data_dict[time_obj][streamer]['diff_count']}\
    \tBot Number: {spect_with_bot[streamer] - len(data_dict[time_obj][streamer]['spect'])}\
    ")

pizfn
    	Viewer Count: 2843    	Spect Count: 2191    	Difference: 652    	Bot Number: 13    
Justees
    	Viewer Count: 1740    	Spect Count: 1320    	Difference: 420    	Bot Number: 11    
Paoloidolo
    	Viewer Count: 1575    	Spect Count: 1337    	Difference: 238    	Bot Number: 17    
Xiuder_
    	Viewer Count: 1261    	Spect Count: 1101    	Difference: 160    	Bot Number: 16    
NanniTwitch
    	Viewer Count: 1001    	Spect Count: 793    	Difference: 208    	Bot Number: 14    
S7ORMyTv
    	Viewer Count: 973    	Spect Count: 717    	Difference: 256    	Bot Number: 14    
Yoshi93
    	Viewer Count: 894    	Spect Count: 707    	Difference: 187    	Bot Number: 30    
Kurolily
    	Viewer Count: 868    	Spect Count: 738    	Difference: 130    	Bot Number: 13    
voghelita
    	Viewer Count: 858    	Spect Count: 668    	Difference: 190    	Bot Number: 30    
NinjoTv
    	Viewer Count: 851    	Spect Count: 665    	Difference: 186    	Bot Number: 9    
Brizz94
    	Viewer Count: 828   

    	Viewer Count: 2    	Spect Count: 0    	Difference: 2    	Bot Number: 10    
BabelCafe
    	Viewer Count: 2    	Spect Count: 0    	Difference: 2    	Bot Number: 6    
greywinner17
    	Viewer Count: 2    	Spect Count: 1    	Difference: 1    	Bot Number: 5    
BIG_GIDAN_TV
    	Viewer Count: 2    	Spect Count: 0    	Difference: 2    	Bot Number: 7    
O7AKU95
    	Viewer Count: 2    	Spect Count: 1    	Difference: 1    	Bot Number: 27    
xlynx_03
    	Viewer Count: 2    	Spect Count: 1    	Difference: 1    	Bot Number: 11    
giannirider789
    	Viewer Count: 2    	Spect Count: 1    	Difference: 1    	Bot Number: 1    
The_Ezlab
    	Viewer Count: 2    	Spect Count: 1    	Difference: 1    	Bot Number: 6    
dadoxtv
    	Viewer Count: 2    	Spect Count: 0    	Difference: 2    	Bot Number: 3    
pir4mid3_
    	Viewer Count: 2    	Spect Count: 0    	Difference: 2    	Bot Number: 8    
Wsnake2015
    	Viewer Count: 2    	Spect Count: 0    	Difference: 2    	Bot Number: 8    
Alicesaolo

### Obtain streamer list and prop

In [11]:
streamer_repeated = []
game_name_repeated = []
viewer_count_repeated = []
spect_count_repeated = []
spect_repeated = []

for istante, value in data_dict.items():
    streamer_repeated.extend(list(value.keys()))
    for streamer, val in value.items():
        game_name_repeated.append(val['game_name'])
        viewer_count_repeated.append(val['viewer_count'])
        spect_count_repeated.append(len(val['spect']))
        spect_repeated.append(val['spect'])

len(streamer_repeated)

197027

In [12]:
streamer_repeated = pd.DataFrame({'streamer': streamer_repeated,
                                  'game_name': game_name_repeated,
                                  'viewer_count': viewer_count_repeated,
                                  'spect_count': spect_count_repeated,
                                  'spect': spect_repeated
                                 })
streamer_repeated

,streamer,game_name,viewer_count,spect_count,spect
0,GrenBaud,Just Chatting,26220,19878,"[vbambola, zruby, ivan_manz, lorenzolacopo, ma..."
1,TheRealMarzaa,Slots,19473,14563,"[ervikingo12, nicolabortoluz, hi_im_wolf_uwu, ..."
2,ilGabbrone,Virtual Casino,6673,4656,"[marcy_081, gabroice, dentico2210, pietrobrusc..."
3,DarioMocciaTwitch,Spore,5549,4480,"[demh95, justniccco, salvatore269, corriere_di..."
4,ocwsport,Sports,5359,3767,"[mrboris89, eltalisman90, robertoverniero, fli..."
...,...,...,...,...,...
197022,Fatalwalker,Teamfight Tactics,0,0,[]
197023,demionso,The Ascent,0,0,[]
197024,emanuelandia,Roblox,0,0,[]
197025,Zacomo,VALORANT,0,0,[]


In [13]:
streamer_grouped = streamer_repeated.groupby("streamer").agg({'viewer_count': ['count', 'mean', 'max'],
                                                              'spect_count': 'mean',
                                                              'spect': 'sum',
                                                              'game_name': (lambda x: list(set(x)))
                                                             })

In [14]:
streamer_grouped.columns = ["_".join(pair) for pair in streamer_grouped.columns]
streamer_grouped['spect_sum'] = streamer_grouped['spect_sum'].apply(set)
streamer_grouped['spect_count_tot'] = streamer_grouped['spect_sum'].apply(len)
streamer_grouped['viewer_count_count'] = streamer_grouped['viewer_count_count'] * interval * freq

streamer_grouped.sort_values('viewer_count_mean', ascending = False, inplace = True)
streamer_grouped

,viewer_count_count,viewer_count_mean,viewer_count_max,spect_count_mean,spect_sum,game_name_<lambda>,spect_count_tot
streamer,,,,,,,
GrenBaud,40,18496.500000,26220,13054.500000,"{romexx__, vbambola, frank2370, zruby, ivan_ma...",[Just Chatting],23502
LyonWGFLive,90,15028.555556,17826,9835.888889,"{paologiovani788, claudio23099, dragon404_ita,...","[Just Chatting, Minecraft]",19458
Tumblurr,140,13510.857143,16339,11122.928571,"{ernestopastaalpesto, sh3va_btw, alangiberti, ...","[Just Chatting, ASMR, Spike Volleyball, Lady i...",31578
TheRealMarzaa,300,12135.633333,22115,9313.566667,"{idestyn14, zruby, lucatos1234, megadodo_92, b...","[Just Chatting, Minecraft, Slots]",46263
ZanoXVII,250,10777.040000,13747,8184.560000,"{alecoma14, ciocco17, diego_sch, tvmarck, teil...","[FIFA 22, Just Chatting, Sports]",49984
...,...,...,...,...,...,...,...
Kinseso,10,0.000000,0,0.000000,{},[Fortnite],0
mohameddoj,10,0.000000,0,0.000000,{},[Just Chatting],0
mohamed73668366,20,0.000000,0,0.000000,{},[Brawl Stars],0


In [15]:
streamer_grouped_cut = streamer_grouped.drop(streamer_grouped[(streamer_grouped.viewer_count_count < min_stream_time)
                                                              | (streamer_grouped.spect_count_tot < min_num_spect_mean)
                                                              | (streamer_grouped.viewer_count_mean < min_num_spect_mean)
                                                             ].index)
streamer_grouped_cut

,viewer_count_count,viewer_count_mean,viewer_count_max,spect_count_mean,spect_sum,game_name_<lambda>,spect_count_tot
streamer,,,,,,,
GrenBaud,40,18496.500000,26220,13054.500000,"{romexx__, vbambola, frank2370, zruby, ivan_ma...",[Just Chatting],23502
LyonWGFLive,90,15028.555556,17826,9835.888889,"{paologiovani788, claudio23099, dragon404_ita,...","[Just Chatting, Minecraft]",19458
Tumblurr,140,13510.857143,16339,11122.928571,"{ernestopastaalpesto, sh3va_btw, alangiberti, ...","[Just Chatting, ASMR, Spike Volleyball, Lady i...",31578
TheRealMarzaa,300,12135.633333,22115,9313.566667,"{idestyn14, zruby, lucatos1234, megadodo_92, b...","[Just Chatting, Minecraft, Slots]",46263
ZanoXVII,250,10777.040000,13747,8184.560000,"{alecoma14, ciocco17, diego_sch, tvmarck, teil...","[FIFA 22, Just Chatting, Sports]",49984
...,...,...,...,...,...,...,...
opbr_hiken,30,3.000000,6,3.666667,"{ddaanni05604, mat_tya, fabiopartenopeo, masse...",[One Piece: Treasure Cruise],7
Ebro_basket_milano,120,3.000000,4,0.750000,"{jo_mari, mammalaura9195, rogrogor}",[Sports],3
irumiao,20,3.000000,3,2.500000,"{udende, ciambella_volante, legnodisequoia}",[Just Chatting],3


In [16]:
columns = ['streamer', 'minutes_live', 'viewer_mean', 'viewer_peak', 'spect_mean', 'spect_list', 'game_name', 'spect_tot']
columns_ordered = ['streamer', 'minutes_live', 'viewer_mean', 'viewer_peak', 'spect_mean', 'spect_tot', 'spect_list', 'game_name']

streamer_dataset = streamer_grouped_cut.copy()
streamer_dataset.reset_index(inplace = True)
streamer_dataset.columns = columns
streamer_dataset = streamer_dataset[columns_ordered]
streamer_dataset['spect_list'] = streamer_dataset['spect_list'].apply(list)
streamer_dataset

,streamer,minutes_live,viewer_mean,viewer_peak,spect_mean,spect_tot,spect_list,game_name
0,GrenBaud,40,18496.500000,26220,13054.500000,23502,"[romexx__, vbambola, frank2370, zruby, ivan_ma...",[Just Chatting]
1,LyonWGFLive,90,15028.555556,17826,9835.888889,19458,"[paologiovani788, claudio23099, dragon404_ita,...","[Just Chatting, Minecraft]"
2,Tumblurr,140,13510.857143,16339,11122.928571,31578,"[ernestopastaalpesto, sh3va_btw, alangiberti, ...","[Just Chatting, ASMR, Spike Volleyball, Lady i..."
3,TheRealMarzaa,300,12135.633333,22115,9313.566667,46263,"[idestyn14, zruby, lucatos1234, megadodo_92, b...","[Just Chatting, Minecraft, Slots]"
4,ZanoXVII,250,10777.040000,13747,8184.560000,49984,"[alecoma14, ciocco17, diego_sch, tvmarck, teil...","[FIFA 22, Just Chatting, Sports]"
...,...,...,...,...,...,...,...,...
4250,opbr_hiken,30,3.000000,6,3.666667,7,"[ddaanni05604, mat_tya, fabiopartenopeo, masse...",[One Piece: Treasure Cruise]
4251,Ebro_basket_milano,120,3.000000,4,0.750000,3,"[jo_mari, mammalaura9195, rogrogor]",[Sports]
4252,irumiao,20,3.000000,3,2.500000,3,"[udende, ciambella_volante, legnodisequoia]",[Just Chatting]
4253,oppoxoppo,120,3.000000,6,1.166667,3,"[connuconebit, birillo_ryan, serghion96_]",[Call Of Duty: Modern Warfare]


### Export Dataset

In [20]:
#relpath3 = '..\DataCollection\Twitch_social_link\\'

#streamer_dataset.to_csv("Datasets/Streamer_dataset.csv")
#with open(f'{relpath3}Streamer_list.txt', 'w', encoding='utf-8') as writeFile:
#    writeFile.writelines("%s\n" % channel for channel in streamer_grouped_cut.index)

## Relations dataset

In [21]:
spect_dict = streamer_grouped_cut[['spect_sum']].to_dict()['spect_sum']
len(spect_dict.keys())

4255

In [22]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Progress')

viewerOverlapDict = {}
completedStreamers = set() #Save which streamers have been processed to avoid repeating
count = 1

for key in spect_dict:
    tempList = {}
    totalLength = len(spect_dict.keys())
    logger.info(str(count) + "/" + str(totalLength)) #Print progress so I can keep track

    for comparisonKey in spect_dict: #Loop through every key again for each key in the dictionary
        if(comparisonKey != key and comparisonKey not in completedStreamers): #If its not a self comparison and the comparison hasn't already been completed
            overlapSize = len(spect_dict[key] & spect_dict[comparisonKey]) #Find the overlap size of the two streamers using set intersection
            if len(spect_dict[key]) < len(spect_dict[comparisonKey]):
                tempList[comparisonKey] = round(overlapSize/len(spect_dict[key])*100, 2) #If the size is over 300 add {comparisonStreamer: overlap} to the dictionary
            else:
                tempList[comparisonKey] = round(overlapSize/len(spect_dict[comparisonKey])*100, 2)
    viewerOverlapDict[key] = tempList #Add this comparison dictionary to the larger dictionary for that streamer
    completedStreamers.add(key) #Add the streamer to completed as no comparisons using this streamer need to be done anymore
    count+=1

INFO:Progress:1/4255
INFO:Progress:2/4255
INFO:Progress:3/4255
INFO:Progress:4/4255
INFO:Progress:5/4255
INFO:Progress:6/4255
INFO:Progress:7/4255
INFO:Progress:8/4255
INFO:Progress:9/4255
INFO:Progress:10/4255
INFO:Progress:11/4255
INFO:Progress:12/4255
INFO:Progress:13/4255
INFO:Progress:14/4255
INFO:Progress:15/4255
INFO:Progress:16/4255
INFO:Progress:17/4255
INFO:Progress:18/4255
INFO:Progress:19/4255
INFO:Progress:20/4255
INFO:Progress:21/4255
INFO:Progress:22/4255
INFO:Progress:23/4255
INFO:Progress:24/4255
INFO:Progress:25/4255
INFO:Progress:26/4255
INFO:Progress:27/4255
INFO:Progress:28/4255
INFO:Progress:29/4255
INFO:Progress:30/4255
INFO:Progress:31/4255
INFO:Progress:32/4255
INFO:Progress:33/4255
INFO:Progress:34/4255
INFO:Progress:35/4255
INFO:Progress:36/4255
INFO:Progress:37/4255
INFO:Progress:38/4255
INFO:Progress:39/4255
INFO:Progress:40/4255
INFO:Progress:41/4255
INFO:Progress:42/4255
INFO:Progress:43/4255
INFO:Progress:44/4255
INFO:Progress:45/4255
INFO:Progress:46/42

INFO:Progress:362/4255
INFO:Progress:363/4255
INFO:Progress:364/4255
INFO:Progress:365/4255
INFO:Progress:366/4255
INFO:Progress:367/4255
INFO:Progress:368/4255
INFO:Progress:369/4255
INFO:Progress:370/4255
INFO:Progress:371/4255
INFO:Progress:372/4255
INFO:Progress:373/4255
INFO:Progress:374/4255
INFO:Progress:375/4255
INFO:Progress:376/4255
INFO:Progress:377/4255
INFO:Progress:378/4255
INFO:Progress:379/4255
INFO:Progress:380/4255
INFO:Progress:381/4255
INFO:Progress:382/4255
INFO:Progress:383/4255
INFO:Progress:384/4255
INFO:Progress:385/4255
INFO:Progress:386/4255
INFO:Progress:387/4255
INFO:Progress:388/4255
INFO:Progress:389/4255
INFO:Progress:390/4255
INFO:Progress:391/4255
INFO:Progress:392/4255
INFO:Progress:393/4255
INFO:Progress:394/4255
INFO:Progress:395/4255
INFO:Progress:396/4255
INFO:Progress:397/4255
INFO:Progress:398/4255
INFO:Progress:399/4255
INFO:Progress:400/4255
INFO:Progress:401/4255
INFO:Progress:402/4255
INFO:Progress:403/4255
INFO:Progress:404/4255
INFO:Progre

INFO:Progress:719/4255
INFO:Progress:720/4255
INFO:Progress:721/4255
INFO:Progress:722/4255
INFO:Progress:723/4255
INFO:Progress:724/4255
INFO:Progress:725/4255
INFO:Progress:726/4255
INFO:Progress:727/4255
INFO:Progress:728/4255
INFO:Progress:729/4255
INFO:Progress:730/4255
INFO:Progress:731/4255
INFO:Progress:732/4255
INFO:Progress:733/4255
INFO:Progress:734/4255
INFO:Progress:735/4255
INFO:Progress:736/4255
INFO:Progress:737/4255
INFO:Progress:738/4255
INFO:Progress:739/4255
INFO:Progress:740/4255
INFO:Progress:741/4255
INFO:Progress:742/4255
INFO:Progress:743/4255
INFO:Progress:744/4255
INFO:Progress:745/4255
INFO:Progress:746/4255
INFO:Progress:747/4255
INFO:Progress:748/4255
INFO:Progress:749/4255
INFO:Progress:750/4255
INFO:Progress:751/4255
INFO:Progress:752/4255
INFO:Progress:753/4255
INFO:Progress:754/4255
INFO:Progress:755/4255
INFO:Progress:756/4255
INFO:Progress:757/4255
INFO:Progress:758/4255
INFO:Progress:759/4255
INFO:Progress:760/4255
INFO:Progress:761/4255
INFO:Progre

INFO:Progress:1073/4255
INFO:Progress:1074/4255
INFO:Progress:1075/4255
INFO:Progress:1076/4255
INFO:Progress:1077/4255
INFO:Progress:1078/4255
INFO:Progress:1079/4255
INFO:Progress:1080/4255
INFO:Progress:1081/4255
INFO:Progress:1082/4255
INFO:Progress:1083/4255
INFO:Progress:1084/4255
INFO:Progress:1085/4255
INFO:Progress:1086/4255
INFO:Progress:1087/4255
INFO:Progress:1088/4255
INFO:Progress:1089/4255
INFO:Progress:1090/4255
INFO:Progress:1091/4255
INFO:Progress:1092/4255
INFO:Progress:1093/4255
INFO:Progress:1094/4255
INFO:Progress:1095/4255
INFO:Progress:1096/4255
INFO:Progress:1097/4255
INFO:Progress:1098/4255
INFO:Progress:1099/4255
INFO:Progress:1100/4255
INFO:Progress:1101/4255
INFO:Progress:1102/4255
INFO:Progress:1103/4255
INFO:Progress:1104/4255
INFO:Progress:1105/4255
INFO:Progress:1106/4255
INFO:Progress:1107/4255
INFO:Progress:1108/4255
INFO:Progress:1109/4255
INFO:Progress:1110/4255
INFO:Progress:1111/4255
INFO:Progress:1112/4255
INFO:Progress:1113/4255
INFO:Progress:11

INFO:Progress:1415/4255
INFO:Progress:1416/4255
INFO:Progress:1417/4255
INFO:Progress:1418/4255
INFO:Progress:1419/4255
INFO:Progress:1420/4255
INFO:Progress:1421/4255
INFO:Progress:1422/4255
INFO:Progress:1423/4255
INFO:Progress:1424/4255
INFO:Progress:1425/4255
INFO:Progress:1426/4255
INFO:Progress:1427/4255
INFO:Progress:1428/4255
INFO:Progress:1429/4255
INFO:Progress:1430/4255
INFO:Progress:1431/4255
INFO:Progress:1432/4255
INFO:Progress:1433/4255
INFO:Progress:1434/4255
INFO:Progress:1435/4255
INFO:Progress:1436/4255
INFO:Progress:1437/4255
INFO:Progress:1438/4255
INFO:Progress:1439/4255
INFO:Progress:1440/4255
INFO:Progress:1441/4255
INFO:Progress:1442/4255
INFO:Progress:1443/4255
INFO:Progress:1444/4255
INFO:Progress:1445/4255
INFO:Progress:1446/4255
INFO:Progress:1447/4255
INFO:Progress:1448/4255
INFO:Progress:1449/4255
INFO:Progress:1450/4255
INFO:Progress:1451/4255
INFO:Progress:1452/4255
INFO:Progress:1453/4255
INFO:Progress:1454/4255
INFO:Progress:1455/4255
INFO:Progress:14

...

INFO:Progress:2783/4255
INFO:Progress:2784/4255
INFO:Progress:2785/4255
INFO:Progress:2786/4255
INFO:Progress:2787/4255
INFO:Progress:2788/4255
INFO:Progress:2789/4255
INFO:Progress:2790/4255
INFO:Progress:2791/4255
INFO:Progress:2792/4255
INFO:Progress:2793/4255
INFO:Progress:2794/4255
INFO:Progress:2795/4255
INFO:Progress:2796/4255
INFO:Progress:2797/4255
INFO:Progress:2798/4255
INFO:Progress:2799/4255
INFO:Progress:2800/4255
INFO:Progress:2801/4255
INFO:Progress:2802/4255
INFO:Progress:2803/4255
INFO:Progress:2804/4255
INFO:Progress:2805/4255
INFO:Progress:2806/4255
INFO:Progress:2807/4255
INFO:Progress:2808/4255
INFO:Progress:2809/4255
INFO:Progress:2810/4255
INFO:Progress:2811/4255
INFO:Progress:2812/4255
INFO:Progress:2813/4255
INFO:Progress:2814/4255
INFO:Progress:2815/4255
INFO:Progress:2816/4255
INFO:Progress:2817/4255
INFO:Progress:2818/4255
INFO:Progress:2819/4255
INFO:Progress:2820/4255
INFO:Progress:2821/4255
INFO:Progress:2822/4255
INFO:Progress:2823/4255
INFO:Progress:28

INFO:Progress:3125/4255
INFO:Progress:3126/4255
INFO:Progress:3127/4255
INFO:Progress:3128/4255
INFO:Progress:3129/4255
INFO:Progress:3130/4255
INFO:Progress:3131/4255
INFO:Progress:3132/4255
INFO:Progress:3133/4255
INFO:Progress:3134/4255
INFO:Progress:3135/4255
INFO:Progress:3136/4255
INFO:Progress:3137/4255
INFO:Progress:3138/4255
INFO:Progress:3139/4255
INFO:Progress:3140/4255
INFO:Progress:3141/4255
INFO:Progress:3142/4255
INFO:Progress:3143/4255
INFO:Progress:3144/4255
INFO:Progress:3145/4255
INFO:Progress:3146/4255
INFO:Progress:3147/4255
INFO:Progress:3148/4255
INFO:Progress:3149/4255
INFO:Progress:3150/4255
INFO:Progress:3151/4255
INFO:Progress:3152/4255
INFO:Progress:3153/4255
INFO:Progress:3154/4255
INFO:Progress:3155/4255
INFO:Progress:3156/4255
INFO:Progress:3157/4255
INFO:Progress:3158/4255
INFO:Progress:3159/4255
INFO:Progress:3160/4255
INFO:Progress:3161/4255
INFO:Progress:3162/4255
INFO:Progress:3163/4255
INFO:Progress:3164/4255
INFO:Progress:3165/4255
INFO:Progress:31

INFO:Progress:3467/4255
INFO:Progress:3468/4255
INFO:Progress:3469/4255
INFO:Progress:3470/4255
INFO:Progress:3471/4255
INFO:Progress:3472/4255
INFO:Progress:3473/4255
INFO:Progress:3474/4255
INFO:Progress:3475/4255
INFO:Progress:3476/4255
INFO:Progress:3477/4255
INFO:Progress:3478/4255
INFO:Progress:3479/4255
INFO:Progress:3480/4255
INFO:Progress:3481/4255
INFO:Progress:3482/4255
INFO:Progress:3483/4255
INFO:Progress:3484/4255
INFO:Progress:3485/4255
INFO:Progress:3486/4255
INFO:Progress:3487/4255
INFO:Progress:3488/4255
INFO:Progress:3489/4255
INFO:Progress:3490/4255
INFO:Progress:3491/4255
INFO:Progress:3492/4255
INFO:Progress:3493/4255
INFO:Progress:3494/4255
INFO:Progress:3495/4255
INFO:Progress:3496/4255
INFO:Progress:3497/4255
INFO:Progress:3498/4255
INFO:Progress:3499/4255
INFO:Progress:3500/4255
INFO:Progress:3501/4255
INFO:Progress:3502/4255
INFO:Progress:3503/4255
INFO:Progress:3504/4255
INFO:Progress:3505/4255
INFO:Progress:3506/4255
INFO:Progress:3507/4255
INFO:Progress:35

INFO:Progress:3809/4255
INFO:Progress:3810/4255
INFO:Progress:3811/4255
INFO:Progress:3812/4255
INFO:Progress:3813/4255
INFO:Progress:3814/4255
INFO:Progress:3815/4255
INFO:Progress:3816/4255
INFO:Progress:3817/4255
INFO:Progress:3818/4255
INFO:Progress:3819/4255
INFO:Progress:3820/4255
INFO:Progress:3821/4255
INFO:Progress:3822/4255
INFO:Progress:3823/4255
INFO:Progress:3824/4255
INFO:Progress:3825/4255
INFO:Progress:3826/4255
INFO:Progress:3827/4255
INFO:Progress:3828/4255
INFO:Progress:3829/4255
INFO:Progress:3830/4255
INFO:Progress:3831/4255
INFO:Progress:3832/4255
INFO:Progress:3833/4255
INFO:Progress:3834/4255
INFO:Progress:3835/4255
INFO:Progress:3836/4255
INFO:Progress:3837/4255
INFO:Progress:3838/4255
INFO:Progress:3839/4255
INFO:Progress:3840/4255
INFO:Progress:3841/4255
INFO:Progress:3842/4255
INFO:Progress:3843/4255
INFO:Progress:3844/4255
INFO:Progress:3845/4255
INFO:Progress:3846/4255
INFO:Progress:3847/4255
INFO:Progress:3848/4255
INFO:Progress:3849/4255
INFO:Progress:38

INFO:Progress:4151/4255
INFO:Progress:4152/4255
INFO:Progress:4153/4255
INFO:Progress:4154/4255
INFO:Progress:4155/4255
INFO:Progress:4156/4255
INFO:Progress:4157/4255
INFO:Progress:4158/4255
INFO:Progress:4159/4255
INFO:Progress:4160/4255
INFO:Progress:4161/4255
INFO:Progress:4162/4255
INFO:Progress:4163/4255
INFO:Progress:4164/4255
INFO:Progress:4165/4255
INFO:Progress:4166/4255
INFO:Progress:4167/4255
INFO:Progress:4168/4255
INFO:Progress:4169/4255
INFO:Progress:4170/4255
INFO:Progress:4171/4255
INFO:Progress:4172/4255
INFO:Progress:4173/4255
INFO:Progress:4174/4255
INFO:Progress:4175/4255
INFO:Progress:4176/4255
INFO:Progress:4177/4255
INFO:Progress:4178/4255
INFO:Progress:4179/4255
INFO:Progress:4180/4255
INFO:Progress:4181/4255
INFO:Progress:4182/4255
INFO:Progress:4183/4255
INFO:Progress:4184/4255
INFO:Progress:4185/4255
INFO:Progress:4186/4255
INFO:Progress:4187/4255
INFO:Progress:4188/4255
INFO:Progress:4189/4255
INFO:Progress:4190/4255
INFO:Progress:4191/4255
INFO:Progress:41

In [20]:
prova = 'ilGabbrone'

print(max(viewerOverlapDict[prova], key=viewerOverlapDict[prova].get))
print(viewerOverlapDict[prova][max(viewerOverlapDict[prova], key=viewerOverlapDict[prova].get)])

n1sc_00
66.67


### Export Dict

In [28]:
#import gzip
#with gzip.open(f'Datasets/Relations_dict.json.gz', 'w') as writefile:
#    writefile.write(json.dumps(viewerOverlapDict).encode('utf-8'))